In [ ]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI API client with API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

# Constants
OPENAI_API_KEY = openai_api_key
VECTOR_STORE_ID = "place your vector store id here"
UPLOAD_URL = "https://api.openai.com/v1/files"
BATCH_URL = f"https://api.openai.com/v1/vector_stores/{VECTOR_STORE_ID}/file_batches"

# Headers for API requests
HEADERS = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v2"
}

def get_markdown_files_from_docs():
    # Get all markdown files from the 'docs' directory
    docs_dir = os.path.join(os.getcwd(), 'docs')
    file_paths = [os.path.join(docs_dir, f) for f in os.listdir(docs_dir) if f.endswith('.md')]
    return file_paths

def upload_files_to_openai(file_paths):
    file_ids = []
    for file_path in file_paths:
        filename = os.path.basename(file_path)  # Extract just the filename
        with open(file_path, 'rb') as f:
            response = requests.post(
                UPLOAD_URL,
                headers={
                    "Authorization": f"Bearer {OPENAI_API_KEY}",
                },
                files={
                    'file': (filename, f, 'text/markdown')
                },
                data={
                    'purpose': 'user_data'  # Change this if another purpose fits better
                }
            )
            if response.status_code == 200:
                file_id = response.json()['id']
                file_ids.append(file_id)
                print(f"Uploaded {filename} successfully, file_id: {file_id}")
            else:
                print(f"Failed to upload {filename}: {response.text}")
    return file_ids

def create_vector_store_file_batch(file_ids):
    data = {
        "file_ids": file_ids,
        # Optional: "chunking_strategy": {"type": "auto"}
    }
    response = requests.post(
        BATCH_URL,
        headers=HEADERS,
        json=data
    )
    if response.status_code == 200:
        batch_id = response.json()['id']
        print(f"Vector store file batch created successfully, batch_id: {batch_id}")
    else:
        print(f"Failed to create vector store file batch: {response.text}")

def main():
    # Step 1: Get markdown files from the 'docs' directory
    file_paths = get_markdown_files_from_docs()

    # Step 2: Upload files to OpenAI
    file_ids = upload_files_to_openai(file_paths)

    # Step 3: Create a vector store file batch
    if file_ids:
        create_vector_store_file_batch(file_ids)

if __name__ == "__main__":
    main()
